# Face tracking pipeline

The following example illustrates how to use the `facenet_pytorch` python package to perform face detection and tracking on an image dataset using MTCNN.

In [1]:
from facenet_pytorch import MTCNN
import torch
import numpy as np
#import mmcv
import cv2
from PIL import Image, ImageDraw
from IPython import display
import os,glob

/Users/rianbutala/.pyenv/versions/3.12.4/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Determine if an nvidia GPU is available

In [2]:
device = torch.device('cpu')
print('Running on device: {}'.format(device))

Running on device: cpu


#### Define MTCNN module

Note that, since MTCNN is a collection of neural nets and other code, the device must be passed in the following way to enable copying of objects when needed internally.

See `help(MTCNN)` for more details.

In [3]:
mtcnn = MTCNN(keep_all=True, device=device)

#### Get a sample video

We begin by loading a video with some faces in it. The `mmcv` PyPI package by mmlabs is used to read the video frames (it can be installed with `pip install mmcv`). Frames are then converted to PIL images.

In [6]:
'''video = mmcv.VideoReader('video.mp4')
frames = [Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) for frame in video]

display.Video('video.mp4', width=640)'''

#path = "/Users/rianbutala/Rian's projects/Coding/Facial Recognition/bse_face_recognition/img/train/notrian"
#save_path = "/Users/rianbutala/Rian's projects/Coding/Facial Recognition/bse_face_recognition/cropped/train/notrian"

path = "/Users/rianbutala/Rian's projects/Coding/Facial Recognition/bse_face_recognition/face_pictures/img/train/notrian"
save_path = "/Users/rianbutala/Rian's projects/Coding/Facial Recognition/bse_face_recognition/face_pictures/cropped/train/notrian"


os.chdir(path)

images = []
counter = 0

for file in glob.glob("*.jpeg"):
    with Image.open(file) as img:
        #img.show()
        images.append(file)
        print("parsed file count = " +str(counter))
        counter = counter + 1

for file in glob.glob("*.jpg"):
    with Image.open(file) as img:
        #img.show()
        images.append(file)
        print("parsed file count = " +str(counter))
        counter = counter + 1
for file in glob.glob("*.png"):
    with Image.open(file) as img:
        #img.show()
        images.append(file)
        print("parsed file count = " +str(counter))
        counter = counter + 1

print(type(images[0]))


parsed file count = 0
parsed file count = 1
parsed file count = 2
parsed file count = 3
parsed file count = 4
parsed file count = 5
parsed file count = 6
parsed file count = 7
parsed file count = 8
parsed file count = 9
parsed file count = 10
parsed file count = 11
parsed file count = 12
parsed file count = 13
parsed file count = 14
parsed file count = 15
parsed file count = 16
parsed file count = 17
parsed file count = 18
parsed file count = 19
parsed file count = 20
parsed file count = 21
parsed file count = 22
parsed file count = 23
parsed file count = 24
parsed file count = 25
parsed file count = 26
parsed file count = 27
parsed file count = 28
parsed file count = 29
parsed file count = 30
parsed file count = 31
parsed file count = 32
parsed file count = 33
parsed file count = 34
parsed file count = 35
parsed file count = 36
parsed file count = 37
parsed file count = 38
parsed file count = 39
parsed file count = 40
parsed file count = 41
parsed file count = 42
parsed file count = 4

#### Run video through MTCNN

We iterate through each frame, detect faces, and draw their bounding boxes on the video frames.

In [7]:

imgs =[]
#print('\rTracking frame: {}'.format(i + 1), end='')

# Detect faces
for count,img in enumerate(images):
    os.chdir(path)
    try:
        print(img)
        img = Image.open(img)
        boxes, _ = mtcnn.detect(img)

        # Draw faces
        frame_draw = img.copy()
        draw = ImageDraw.Draw(frame_draw)
        try:
            for box in boxes:
                draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
                print(box)

            cropped_img = img.copy()
            cropped_img = cropped_img.crop(boxes[0])
            #d = display.display(cropped_img, display_id=True)
            os.chdir(save_path)
            cropped_img.save("img_"+"c"+str(count)+".jpeg")
            print("saved!! @ "+ "img_"+"c"+str(count)+".jpeg")


            # Add to frame list
            imgs.append(frame_draw.resize((640, 360), Image.BILINEAR))
            print('\nDone')
        except:
            print("caught!!")
    except:
        print("caught 2")
print("all done!!")

2024-06-17 13:31:17c349.jpeg
[1505.47021484375 229.4049530029297 1771.368408203125 647.5571899414062]
[518.3839721679688 889.3789672851562 542.6439819335938 919.10546875]
saved!! @ img_c0.jpeg

Done
2024-06-13 18:25:46c276.jpeg
[909.7637329101562 70.14430236816406 1264.0540771484375 548.0277709960938]
saved!! @ img_c1.jpeg

Done
2024-07-01 13:49:54c35.jpeg
[1269.7188720703125 529.3096923828125 1493.302001953125 874.5345458984375]
[432.787353515625 944.9677734375 490.3354187011719 1006.50390625]
[503.6223449707031 983.6722412109375 541.0765991210938 1030.765869140625]
saved!! @ img_c2.jpeg

Done
2024-06-13 18:25:43c192.jpeg
[747.0740966796875 195.73544311523438 1099.41259765625 649.9796752929688]
saved!! @ img_c3.jpeg

Done
2024-06-17 13:31:11c227.jpeg
[758.6246337890625 581.7855834960938 998.8807983398438 833.2688598632812]
[1719.404296875 996.7234497070312 1783.19189453125 1058.1390380859375]
[1475.3004150390625 1009.3809814453125 1526.1820068359375
 1071.4439697265625]
saved!! @ img_

In [19]:
'''frames_tracked = []
for i, frame in enumerate(frames):
    print('\rTracking frame: {}'.format(i + 1), end='')
    
    # Detect faces
    boxes, _ = mtcnn.detect(frame)
    
    # Draw faces
    frame_draw = frame.copy()
    draw = ImageDraw.Draw(frame_draw)
    for box in boxes:
        draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)
    
    # Add to frame list
    frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))
print('\nDone')'''

"frames_tracked = []\nfor i, frame in enumerate(frames):\n    print('\rTracking frame: {}'.format(i + 1), end='')\n    \n    # Detect faces\n    boxes, _ = mtcnn.detect(frame)\n    \n    # Draw faces\n    frame_draw = frame.copy()\n    draw = ImageDraw.Draw(frame_draw)\n    for box in boxes:\n        draw.rectangle(box.tolist(), outline=(255, 0, 0), width=6)\n    \n    # Add to frame list\n    frames_tracked.append(frame_draw.resize((640, 360), Image.BILINEAR))\nprint('\nDone')"

#### Display detections

In [20]:
#d = display.display(imgs[0], display_id=True)
#for img in imgs:
#    display.display(img,display_id=True)

In [21]:
'''d = display.display(frames_tracked[0], display_id=True)
i = 1
try:
    while True:
        d.update(frames_tracked[i % len(frames_tracked)])
        i += 1
except KeyboardInterrupt:
    pass'''

'd = display.display(frames_tracked[0], display_id=True)\ni = 1\ntry:\n    while True:\n        d.update(frames_tracked[i % len(frames_tracked)])\n        i += 1\nexcept KeyboardInterrupt:\n    pass'

#### Save tracked video

In [22]:
'''dim = frames_tracked[0].size
fourcc = cv2.VideoWriter_fourcc(*'FMP4')    
video_tracked = cv2.VideoWriter('video_tracked.mp4', fourcc, 25.0, dim)
for frame in frames_tracked:
    video_tracked.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))
video_tracked.release()'''

"dim = frames_tracked[0].size\nfourcc = cv2.VideoWriter_fourcc(*'FMP4')    \nvideo_tracked = cv2.VideoWriter('video_tracked.mp4', fourcc, 25.0, dim)\nfor frame in frames_tracked:\n    video_tracked.write(cv2.cvtColor(np.array(frame), cv2.COLOR_RGB2BGR))\nvideo_tracked.release()"